In [ ]:
'''
Project: Bangkit Capstone Project
Group: B21-CAP0194
Part: Modelling
'''

'\nProject: Bangkit Capstone Project\nGroup: B21-CAP0194\nPart: Modelling\n'

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 32
trunc_type = 'post'
oov_tok = "<OOV>"

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#Import Labelled Dataset
url = 'https://drive.google.com/file/d/1nOn8OONtZA7pcX4Pvh8W_wOUg49ZWTvT/view?usp=sharing'
file_id = url.split('/')[-2]
csv_url = 'https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(csv_url)
df.head()

,index,text,label
0,0,the ironies of the movement are too painfully...,work places
1,1,women in kuwait have launched a social media c...,cyber
2,2,triger warning raising awareness of sexual ass...,private places
3,3,charges brought against the guy that smacked m...,private places
4,4,and the fourth presenter on 26th may is hannah...,educational places


In [ ]:
df_cut = df[:100]

In [ ]:
df_cut.head(50)

,index,text,label
0,0,the ironies of the movement are too painfully...,work places
1,1,women in kuwait have launched a social media c...,cyber
2,2,triger warning raising awareness of sexual ass...,private places
3,3,charges brought against the guy that smacked m...,private places
4,4,and the fourth presenter on 26th may is hannah...,educational places
5,5,this is so messed up are there any men in the...,public places
6,6,to tackle an epidemic of male violence against...,cyber
7,7,there has been a new series of harassment alle...,private places
8,8,want to see what the movement looks like its ...,public places
9,9,female students dont want to have sex w dolan ...,educational places


In [ ]:
import getpass

APIKEY = getpass.getpass()

In [ ]:
# running Translate API
#from googleapiclient.discovery import build
#service = build('translate', 'v2', developerKey=APIKEY)

#df_cut['indo_text'] = df_cut['text'].apply(lambda x: (service.translations().list(source='en', target='id', q=x).execute())).apply(lambda x: x['translations']).apply(lambda x: x[0]).apply(lambda x: x['translatedText'])

In [ ]:
# running Translate API
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

df['indo_text'] = df['text'].apply(lambda x: (service.translations().list(source='en', target='id', q=x).execute())).apply(lambda x: x['translations']).apply(lambda x: x[0]).apply(lambda x: x['translatedText'])

In [ ]:
df['indo_text'].head(50)

In [ ]:
#Save DataFrame to CSV
df_translated = pd.DataFrame()
df_translated['index'] = df['index']
df_translated['text'] = df['text']
df_translated['indo_text'] = df['indo_text']
df_translated.to_csv("capstonetweettranslatednew.csv", index=False)

In [ ]:
labels = pd.get_dummies(df.label)

In [ ]:
df_new = pd.concat([df, labels], axis=1)
df_new = df_new.drop(columns='label')

In [ ]:
tweet = df_new['text'].values
label = df_new[["cyber", "educational places", "neutral", "private places", "public places", "work places"]].values

In [ ]:
label

array([[0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1]], dtype=uint8)

In [ ]:
from sklearn.model_selection import train_test_split
#Train Text, Test Text, Train Label, Test Label
X_train, X_test, y_train, y_test = train_test_split(tweet, label, test_size=0.2, shuffle=True)

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(X_train)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)

In [ ]:
import tensorflow as tf
'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])

'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])


transfer learning models for NLP: BERT, coba buka ini https://www.dlology.com/blog/keras-meets-universal-sentence-encoder-transfer-learning-for-text-data/

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 16)            160000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 128)           41472     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 198       
Total params: 244,966
Trainable params: 244,966
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 50
model.fit(padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_test))

Epoch 1/50
353/353 [==============================] - 10s 17ms/step - loss: 1.5400 - accuracy: 0.3557 - val_loss: 1.2101 - val_accuracy: 0.5344
Epoch 2/50
353/353 [==============================] - 5s 13ms/step - loss: 0.9892 - accuracy: 0.6234 - val_loss: 0.9162 - val_accuracy: 0.6889
Epoch 3/50
353/353 [==============================] - 5s 14ms/step - loss: 0.6224 - accuracy: 0.7804 - val_loss: 0.7863 - val_accuracy: 0.7282
Epoch 4/50
353/353 [==============================] - 5s 14ms/step - loss: 0.3840 - accuracy: 0.8708 - val_loss: 0.7240 - val_accuracy: 0.7870
Epoch 5/50
353/353 [==============================] - 5s 14ms/step - loss: 0.2491 - accuracy: 0.9194 - val_loss: 0.7852 - val_accuracy: 0.7970
Epoch 6/50
353/353 [==============================] - 5s 14ms/step - loss: 0.1780 - accuracy: 0.9482 - val_loss: 0.7652 - val_accuracy: 0.8104
Epoch 7/50
353/353 [==============================] - 5s 14ms/step - loss: 0.1353 - accuracy: 0.9617 - val_loss: 0.7834 - val_accuracy: 0.817

In [ ]:
text = ["i meet my mom and my dad at home"]

tokenized_text = tokenizer.texts_to_sequences(text)

In [ ]:
prediction = model.predict(tokenized_text)

In [ ]:
labels = ["cyber", "educational places", "neutral", "private places", "public places", "work places"]
predicted = prediction.argmax(axis=1)
predict_labels = [labels[predict] for predict in predicted]
print(prediction)
print(predict_labels)

[[6.7048204e-05 5.9439770e-07 6.3853212e-10 9.9992669e-01 9.4983001e-08
  5.5752753e-06]]
['private places']


In [ ]:
#Exporting the Model (TFLITE)
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: saved_model/1/assets


INFO:tensorflow:Assets written to: saved_model/1/assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
#converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpx09_w08h/assets


INFO:tensorflow:Assets written to: /tmp/tmpx09_w08h/assets


In [ ]:
import pathlib

tflite_model_file = pathlib.Path('/content/model.tflite')
tflite_model_file.write_bytes(tflite_model)

1014788

In [ ]:
from google.colab import files
files.download(tflite_model_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Exporting the Tokenized Word
import json

with open( 'word_dict.json' , 'w' ) as file:
    json.dump( tokenizer.word_index , file )